In [ ]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


/opt/anaconda3/envs/crypto-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

1735766992

In [3]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

2025-01-01 23:16:49,764 INFO: Initializing external client
2025-01-01 23:16:49,765 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-01 23:16:51,213 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164448


In [4]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


1732310209.0

In [5]:
today, starting_timestamp

(1735766209, 1732310209.0)

In [6]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.81s) 


,timestamp,open,high,low,close,volume,volume_buy,volume_sell
0,1732320000,257.25,264.02,253.20,255.26,367378.262582,157863.310285,209514.952298
1,1732406400,255.26,259.96,241.21,252.93,333221.510079,173161.319468,160060.190611
2,1732492800,252.93,256.65,231.32,233.96,430081.090503,191964.534235,238116.556267
3,1732579200,233.96,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889
4,1732665600,230.68,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184


In [7]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.02s) 


,timestamp,open
0,1732320000,99000.0
1,1732406400,97759.1
2,1732492800,97994.1
3,1732579200,93000.0
4,1732665600,91914.5


In [8]:
fng_feature_group = fs.get_feature_group('f_n_g_index', version=8)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['fng_classification'] = fng_batch_data['fng_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.59s) 


,fng_value,fng_classification,timestamp
0,93,4,1.732320e+09
1,80,4,1.732406e+09
2,82,4,1.732493e+09
3,79,4,1.732579e+09
4,75,3,1.732666e+09


In [9]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=3,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


/var/folders/vc/103l9n915p1_fcsd6kw28dsm0000gn/T/84e4414c-a29a-4e75-b7ae-aaf9a8639123/solana_price_daily_model_10d/3


<Sequential name=sequential, built=True>

In [10]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

,high,low,close,volume,volume_buy,volume_sell,open_bitcoin,fng_value,fng_classification,open_solana
0,264.02,253.20,255.26,367378.262582,157863.310285,209514.952298,99000.0,93,4,257.25
1,259.96,241.21,252.93,333221.510079,173161.319468,160060.190611,97759.1,80,4,255.26
2,256.65,231.32,233.96,430081.090503,191964.534235,238116.556267,97994.1,82,4,252.93
3,239.68,221.82,230.68,445887.775346,201503.607457,244384.167889,93000.0,79,4,233.96
4,245.00,227.66,242.26,265621.873707,161330.187523,104291.686184,91914.5,75,3,230.68


In [ ]:
scaler_labels = joblib.load('scaler_labels.pkl')
scaler_features = joblib.load('scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


In [17]:
all_data.columns

Index(['high', 'low', 'close', 'volume', 'volume_buy', 'volume_sell',
       'open_bitcoin', 'fng_value', 'fng_classification', 'open_solana'],
      dtype='object')

In [21]:
predictions = model.predict(X)[0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


In [22]:
predictions

array([0.44728172, 0.5388375 , 0.44224438, 0.38249242, 0.43675682,
       0.5306252 , 0.48221654, 0.70167875, 0.40769655, 0.507215  ],
      dtype=float32)

In [23]:
range(len(predictions))

range(0, 10)

In [25]:
timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions))]
timestamps

[1735852609,
 1735939009,
 1736025409,
 1736111809,
 1736198209,
 1736284609,
 1736371009,
 1736457409,
 1736543809,
 1736630209]

In [31]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=1,
    primary_key=['timestamp']
)

In [32]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions
result

,timestamp,open_solana
0,1735852609,0.447282
1,1735939009,0.538837
2,1736025409,0.442244
3,1736111809,0.382492
4,1736198209,0.436757
5,1736284609,0.530625
6,1736371009,0.482217
7,1736457409,0.701679
8,1736543809,0.407697
9,1736630209,0.507215


In [33]:
monitor_fg.insert(result)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164448/fs/1155151/fg/1394446


Uploading Dataframe: 100.00% |██████████| Rows 10/10 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164448/jobs/named/solana_predictions_1_offline_fg_materialization/executions


(Job('solana_predictions_1_offline_fg_materialization', 'SPARK'), None)